After exploring the data on the small database it is time to check if it applies to the main database and adjust it accordingly.

In [1]:
import pandas as pd
import numpy as np

#Copying the functions used previously
def apply_median(df,columns):
    '''takes in a list of columns from a dataframe (df) and applies the median of that column where the value is a NaN'''
    for col in columns:
        df.loc[:,col] = df.loc[:,col].fillna(df.loc[:,col].median())
    return df;


def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''
    for col in  cat_cols:
        try:
            #for each cat add dummy var, drop original column
            df = pd.concat([df.drop(columns=col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df;



def categorical_months_to_years (df,column):
    '''
    INPUT:
    df - pandas dataframe
    column -  numerical column in months to be transformed into categorical columns with integer intervals in years.
    
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains new columns with the integer intervals found in the maximum years value.
            2. removes the original column.
            3. dummy columns for each of the intervals found within the maximum year value.
            4. Use a prefix of the column name with an underscore (_) for separating 
    '''
    
    
    months = 12 # number of months to divide
    
    #finding the maximum interval
    max_years = int(df.loc[:,column[0]].dropna().max()/months)
    
    #creating the name for the maximum interval column
    max_column_list = column[0]+'_gt_'+str(max_years)#gt = greater than
    
    #setting the value to start the cycle
    years = max_years
    
    #creating a list of the years
    range_years = [max_years]
    
    #creating a list with the column names
    column_list = [max_column_list]
    
    #creating a list all the integer intervals within the maximum range and list of strings with the respective column names
    for i in range (0,max_years):
        if years > 1:
            years = years - 1
            range_years.append(years)
            column_list.append(column[0]+'_lt_'+str(years+1)+'_gt_'+str(years))
        else:
            range_years.append(0)
            column_list.append(column[0]+'_lt_'+str(years))#lt = less than

    #creating a temporary empty dataframe with the same number of rows as df and the column list
    temp_df = pd.DataFrame(index=range(0,df.shape[0]), columns=column_list)
    
    #array with the index of intervals where the df column value is in 
    intervals = np.digitize(df.loc[:,column[0]].values, bins=range_years)
    
    #filling the respective interval column with 1 and the remaning with 0
    for i in range(0, temp_df.shape[0]):
        index = intervals[i]
        temp_df.iloc[i,index] = 1
        temp_df.loc[i, temp_df.columns != column_list[index]]= 0
    
    #adding the new columns to the original dataframe and dropping the original column
    df = pd.concat([df.drop(columns=column, axis=1), temp_df], axis=1)
    
    return df;


def categorical_1k_wNaN (df,column):
    '''
    INPUT:
    df - pandas dataframe
    column -  numerical column to be transform into categorical columns
    
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains new columns with the 1000 intervals found in the maximum value.
            2. removes the original column.
            3. dummy columns for each of the intervals found within the maximum year value.
            4. 0,1 column where 1 represents the value being a NaN in the original column
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''
    
    
    thousands = 1000 # number of thousands to divide
    
    #finding the maximum interval
    max_thousands = int(df.loc[:,column[0]].dropna().max()/thousands)
    
    max_interval = int((df.loc[:,column[0]].dropna().max())/thousands)*thousands
    
    #creating the name for the maximum interval column
    max_column_list = column[0]+'_gt_'+str(max_thousands)+'k'#gt = greater than
    
    #setting the value to start the cycle
    salary = max_interval
    
    #creating a list of the salary
    range_salary = [max_interval]
    
    #creating a list with the column names
    column_list = [max_column_list]
    
    #creating a list with all the thousands intervals within the maximum range and list of strings with the respective column names
    for i in range (0,max_thousands+2):#+2 because of the 500 interval between 0 and 1k and the NaN column
        if salary > 1000:
            salary = salary - 1000
            range_salary.append(salary)
            column_list.append(column[0]+'_lt_'+str(salary/1000+1)+'k'+'_gt_'+str(salary/1000)+'k')
        elif salary > 500:
            salary = salary - 500
            range_salary.append(salary)
            column_list.append(column[0]+'_lt_'+str(salary/1000+0.5)+'k'+'_gt_'+str(salary/1000)+'k')
        elif salary <= 500 and salary > 0:
            range_salary.append(0)
            column_list.append(column[0]+'_lt_'+str(salary/1000))#lt = less than
            salary = salary - 500
        else:
            column_list.append(column[0]+'_NaN')

    #creating a temporary empty dataframe with the same number of rows as df and the column list
    temp_df = pd.DataFrame(index=range(0,df.shape[0]), columns=column_list)
    
    #array with the index of intervals created where the column value is in 
    intervals = np.digitize(df.loc[:,column[0]].values, bins=range_salary)
    
    #filling the NaN column in temp_df with 0,1 according to the presence of NaNs in the column
    for i in range(0, df.shape[0]):
        if pd.isnull(df.loc[i,column[0]]):
            temp_df.loc[i,column_list[-1]] = 1
            temp_df.loc[i, temp_df.columns != column_list[-1]]= 0
        else:
            temp_df.loc[i,column_list[-1]] = 0
    
    #for the non NaN columns filling the respective interval column with 1 and the remaning with 0
    for i in range(0, temp_df.shape[0]):
        if temp_df.loc[i,column_list[-1]] == 0:
            index = intervals[i]
            temp_df.iloc[i,index] = 1
            temp_df.loc[i, temp_df.columns != column_list[index]]= 0
    
    #adding the new columns to the original dataframe and dropping the original column
    df = pd.concat([df.drop(columns=column, axis=1), temp_df], axis=1)
    
    return df;


In [2]:
def transform_data(df):
    '''
    Input
    A portfolio type dataframe
    
    Output
    Pandas dataframe with no string type variables and no NaN values
    '''
    
    #eliminating all columns with less than 50% of the values
    less50_cols = set(df.loc[:, df.isna().mean() > .5])
    df = df.drop(columns=less50_cols)
    
    #list of columns to be dropped
    columns_drop = ['qt_socios_pf','qt_socios_pj', 'fl_matriz','natureza_juridica_macro',
                    'de_natureza_juridica','de_ramo','idade_emp_cat', 'dt_situacao','fl_st_especial',
                    'fl_email','fl_telefone','nm_segmento','fl_optante_simples','nm_micro_regiao','sg_uf_matriz',
                    'fl_optante_simei','vl_faturamento_estimado_aux','vl_faturamento_estimado_grupo_aux']
    
    #dropping the columns from the list of columns to be dropped
    df = df.drop(columns=columns_drop)
    
    #quick function to transform True into 1 and False into 0
    transform_boolean = lambda col: 1 if col == True else 0
    
    #list of columns to apply the funcion above
    boolean_cols = ['fl_me','fl_sa','fl_epp','fl_mei','fl_ltda','fl_rm','fl_spa',
                    'fl_antt','fl_veiculo','fl_simples_irregular','fl_passivel_iss']
    
    #applying the transform boolean function to the list of columns above
    for col in boolean_cols:
        df[col] = df.loc[:,col].apply(transform_boolean)
    
    #quick function to transform having vehicles into 1 and not having vehicles into 0
    transform_vehicles = lambda col: 0 if col == 0 else 1
    
    #list of vehicles columns
    vehicle_cols = ['vl_total_veiculos_leves_grupo','vl_total_veiculos_pesados_grupo']
    
    #applying the transform_vehicles function to the above list
    for col in vehicle_cols:
        df[col] = df.loc[:,col].apply(transform_vehicles)
    
    #list of columns to fill the NaN values with the median
    median_cols = ['idade_media_socios','idade_maxima_socios','idade_minima_socios','qt_socios','qt_socios_st_regular']
    
    #filling the NaN values with the median of the column in the list above
    apply_median(df,median_cols)
    
    #rounding to 2 decimal cases the idade_empresa_anos feature
    df.loc[:,'idade_empresa_anos'] = df.loc[:,'idade_empresa_anos'].round(decimals=2)
    
    #replacing the no information string with a NaN value in the de_faixa_faturamento_estimado feature
    df.loc[:,'de_faixa_faturamento_estimado'] = df.loc[:,'de_faixa_faturamento_estimado'].replace('SEM INFORMACAO', np.nan)
    
    #list of categorical columns to transform into dummy type columns
    #dummy_cols = ['sg_uf','setor','nm_divisao']
    
    #transforming the list of columns above into dummy type columns
    #df = create_dummy_df(df,dummy_cols,False)
    
    #list of categorical columns to transform into dummy type columns with NaN as a feature
    dummy_cols_NA_True = ['sg_uf','setor','nm_divisao','de_saude_tributaria','de_nivel_atividade','nm_meso_regiao',
                          'de_faixa_faturamento_estimado','de_faixa_faturamento_estimado_grupo']
    
    #transforming the list of columns above into dummy type columns with NaN as a feature
    df = create_dummy_df(df,dummy_cols_NA_True,True)
    
    #Transforming the numerical columns below into categorical type features
    #year_column=['nu_meses_rescencia']
    #df = categorical_months_to_years(df,year_column)
    
    #salary = ['empsetorcensitariofaixarendapopulacao']
    #df = categorical_1k_wNaN(df,salary)
    
    
    return df;

Before starting to apply our ETL to the main database a little side note: The functions designed to transform numerical into categorical data written before were generating too many labels. Which means the features **nu_meses_rescencia** and **empsetorcensitariofaixarendapopulacao** need to be rethought. But for now we'll comment that part of the code and move on to further inspect the main database.

In [3]:
def load_data(df,filepath):
    '''
    Input
    pandas dataframe and the filepath to save the dataframe
    Output
    File with the pandas dataframe
    '''
    df.to_csv(filepath, index = False)

In [5]:
filepath = 'data/estaticos_market.csv'

def extract_data(filepath):
    '''
    Input
    filepath - string of the file path where the csv is located
    Output - pandas dataframe
    '''
    # read the large csv file with specified chunksize 
    df_chunk = pd.read_csv(filepath, chunksize=10000, index_col=0)
    
    chunk_list = []  # append each chunk df here 
    
    # Each chunk is in df format
    for chunk in df_chunk:  
        # perform data filtering 
        chunk_filter = transform_data(chunk)
        
        # Once the data filtering is done, append the chunk to list
        chunk_list.append(chunk_filter)
        
        
    # concat the list into dataframe 
    df_concat = pd.concat(chunk_list)
    
    return df_concat;

market = extract_data(filepath)
market.shape #we know that our database has 462298 so this number serves to check if the import is working.

(462298, 173)

In [6]:
market.head()

,id,idade_empresa_anos,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,fl_rm,fl_spa,fl_antt,...,"de_faixa_faturamento_estimado_grupo_DE R$ 81.000,01 A R$ 360.000,00",de_faixa_faturamento_estimado_grupo_nan,nm_divisao_ATIVIDADES DE APOIO A EXTRACAO DE MINERAIS,nm_divisao_DESCONTAMINACAO E OUTROS SERVICOS DE GESTAO DE RESIDUOS,nm_divisao_EXTRACAO DE PETROLEO E GAS NATURAL,nm_divisao_FABRICACAO DE PRODUTOS FARMOQUIMICOS E FARMACEUTICOS,"de_faixa_faturamento_estimado_ATE R$ 81.000,00","de_faixa_faturamento_estimado_DE R$ 500.000.000,01 A 1 BILHAO DE REAIS",nm_divisao_EXTRACAO DE CARVAO MINERAL,nm_divisao_FABRICACAO DE PRODUTOS DO FUMO
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,14.46,0,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,1.46,0,0,0,1,0,0,0,0,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,7.09,0,0,0,1,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,6.51,0,0,0,0,0,0,0,0,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,3.20,0,0,0,0,0,0,0,0,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see that there are NaNs in the database, so we'll explore what went wrong. First let's take a look at the labels that were generated and then we'll see which ones have NaN values.

In [7]:
list(market.columns)

['id',
 'idade_empresa_anos',
 'fl_me',
 'fl_sa',
 'fl_epp',
 'fl_mei',
 'fl_ltda',
 'fl_rm',
 'fl_spa',
 'fl_antt',
 'fl_veiculo',
 'vl_total_veiculos_pesados_grupo',
 'vl_total_veiculos_leves_grupo',
 'de_saude_rescencia',
 'nu_meses_rescencia',
 'fl_simples_irregular',
 'empsetorcensitariofaixarendapopulacao',
 'fl_passivel_iss',
 'qt_socios',
 'idade_media_socios',
 'idade_maxima_socios',
 'idade_minima_socios',
 'qt_socios_st_regular',
 'qt_filiais',
 'sg_uf_AM',
 'sg_uf_MA',
 'sg_uf_PI',
 'sg_uf_RN',
 'sg_uf_RO',
 'sg_uf_nan',
 'setor_COMERCIO',
 'setor_CONSTRUÇÃO CIVIL',
 'setor_INDUSTRIA',
 'setor_SERVIÇO',
 'setor_nan',
 'nm_divisao_AGENCIAS DE VIAGENS OPERADORES TURISTICOS E SERVICOS DE RESERVAS',
 'nm_divisao_AGRICULTURA PECUARIA E SERVICOS RELACIONADOS',
 'nm_divisao_ALIMENTACAO',
 'nm_divisao_ALOJAMENTO',
 'nm_divisao_ALUGUEIS NAO IMOBILIARIOS E GESTAO DE ATIVOS INTANGIVEIS NAO FINANCEIROS',
 'nm_divisao_ARMAZENAMENTO E ATIVIDADES AUXILIARES DOS TRANSPORTES',
 'nm_divisao_

In [10]:
set(market.loc[:, market.isna().mean() > .5])

{'de_faixa_faturamento_estimado_ATE R$ 81.000,00',
 'de_faixa_faturamento_estimado_DE R$ 500.000.000,01 A 1 BILHAO DE REAIS',
 'nm_divisao_DESCONTAMINACAO E OUTROS SERVICOS DE GESTAO DE RESIDUOS',
 'nm_divisao_EXTRACAO DE CARVAO MINERAL',
 'nm_divisao_FABRICACAO DE PRODUTOS DO FUMO',
 'nm_divisao_FABRICACAO DE PRODUTOS FARMOQUIMICOS E FARMACEUTICOS',
 'nm_divisao_ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUICOES EXTRATERRITORIAIS'}

In [11]:
market.isna().mean().sort_values(ascending=False)[0:50]

nm_divisao_EXTRACAO DE CARVAO MINERAL                                                                0.978369
nm_divisao_FABRICACAO DE PRODUTOS DO FUMO                                                            0.956738
nm_divisao_ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUICOES EXTRATERRITORIAIS                         0.843612
nm_divisao_DESCONTAMINACAO E OUTROS SERVICOS DE GESTAO DE RESIDUOS                                   0.762058
nm_divisao_FABRICACAO DE PRODUTOS FARMOQUIMICOS E FARMACEUTICOS                                      0.653903
de_faixa_faturamento_estimado_ATE R$ 81.000,00                                                       0.589010
de_faixa_faturamento_estimado_DE R$ 500.000.000,01 A 1 BILHAO DE REAIS                               0.567379
nm_divisao_EXTRACAO DE PETROLEO E GAS NATURAL                                                        0.459223
nm_divisao_ATIVIDADES DE APOIO A EXTRACAO DE MINERAIS                                                0.372699
nm_divisao

In [12]:
market['nm_divisao_EXTRACAO DE CARVAO MINERAL'].value_counts()

0.0    9999
1.0       1
Name: nm_divisao_EXTRACAO DE CARVAO MINERAL, dtype: int64

In [13]:
market['nm_divisao_DESCONTAMINACAO E OUTROS SERVICOS DE GESTAO DE RESIDUOS'].value_counts()

0.0    109988
1.0        12
Name: nm_divisao_DESCONTAMINACAO E OUTROS SERVICOS DE GESTAO DE RESIDUOS, dtype: int64

In [14]:
market['nm_divisao_ATIVIDADES DE PRESTACAO DE SERVICOS DE INFORMACAO'].value_counts()

0    461549
1       749
Name: nm_divisao_ATIVIDADES DE PRESTACAO DE SERVICOS DE INFORMACAO, dtype: int64

So besides the two features we opted to comment (see above), the features that have NaNs are the dummies we created. We could simply apply a replace NaN with 0 to all of these new features and fix this.

Now let's explore the commented features above to see what we can do.

In [15]:
list(market['empsetorcensitariofaixarendapopulacao'].sort_values(ascending=False))[0:100]

[75093.84,
 75093.84,
 75093.84,
 75093.84,
 75093.84,
 75093.84,
 75093.84,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 30861.81,
 15137.49,
 15137.49,
 15137.49,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 13793.0,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,
 12512.93,

In [18]:
market[market['empsetorcensitariofaixarendapopulacao']>10000].shape[0] #0.1% (462/462298)

462

In [19]:
market[market['empsetorcensitariofaixarendapopulacao']>5000].shape[0] #1.1% (5037/462298)

5037

In [20]:
market[market['empsetorcensitariofaixarendapopulacao']>2500].shape[0]#7.2% (33139/462298)

33139

In [21]:
market[market['empsetorcensitariofaixarendapopulacao']>1000].shape[0] #32,12%  (148511/462298)

148511

In the dictionary this feature is described as the average salary of the population. In 2019 less than 50% of the population received more than the minimum, 1k, which means that 32% above 1k is realistic, and when we look at the % of the top values 1% above 5k we could even consider outliers. However such high values as 30 and 75k raise suspicious about how is this being colected and what it truly is. And since it is a feature that discloses about the enviroment the company is in instead of direct information about it, before knowing more about this feature we'll drop it and come back to it in the future.

In [22]:
market['de_saude_rescencia'].value_counts()

ACIMA DE 1 ANO    378896
ATE 1 ANO          38101
SEM INFORMACAO     30425
ATE 3 MESES           13
ATE 6 MESES           12
Name: de_saude_rescencia, dtype: int64

In [24]:
market['nu_meses_rescencia'].value_counts().sort_values(ascending=False)

23.0    108609
22.0     89143
24.0     44574
48.0     26480
25.0     25087
26.0     24218
27.0     18888
21.0     12154
7.0      10016
47.0      9637
9.0       9034
50.0      6562
10.0      6377
8.0       5811
11.0      4418
49.0      3999
20.0      2881
12.0      2445
19.0      2346
54.0      1560
52.0       847
28.0       487
18.0       474
41.0       192
39.0       151
40.0       125
38.0        52
53.0        51
51.0        45
66.0        41
17.0        38
65.0        33
36.0        33
13.0        32
14.0        31
29.0        21
15.0        19
35.0        18
37.0        17
16.0        14
42.0        10
46.0         8
5.0          8
31.0         6
45.0         5
2.0          5
6.0          4
64.0         4
0.0          4
30.0         4
1.0          4
Name: nu_meses_rescencia, dtype: int64

In the previous attempt we tried to get more information on this time feature. But as we talked about it before we can just use the **de_saude_rescencia** feature as categorical feature and replace the NaNs with SEM INFORMACAO (no information). So in order to simply things we are going to chose this path for now.

To do in the Market ETL:
df['de_saude_rescencia'] = df['de_saude_rescencia'].fillna('SEM INFORMACAO')
cat_cols = ['de_saude_rescencia']
create_dummy_df(df, cat_cols, False)

drop = [nu_meses_rescencia,empsetorcensitariofaixarendapopulacao]

replace the nan values in the categorical columns with zeros